# Sustainable retirement age
AIM: assuming that the current situation of working / retired people is sustainable, and keeping that as reference, what is the retirement age for the whole century, considering the forecast model of the population I have built in my previous Notebook? 

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import json

import plotly.io as pio
pio.renderers.default = 'notebook'

pd.options.plotting.backend = "plotly"

In [2]:
# read json 
with open("data/3_scenario_labels.json", "r") as f:
    scenario_labels = json.load(f)

dfp_projs = {}
for scenario_label in scenario_labels:
    df = pd.read_csv(f"data/3_population_projection_{scenario_label}.csv", index_col=0)
    df.columns = df.columns.astype(int)
    dfp_projs[scenario_label] = df

In [3]:
# I'm making some assumptions on the average age of retirement and start working
dfp = pd.read_csv("data/1_population_by_age_year.csv", index_col=0)
dfp.columns = dfp.columns.astype(int)

first_year = dfp.columns[0]
last_year = dfp.columns[-1]

working_age = { y: 20 + 2*(y-first_year)/(last_year-first_year) for y in dfp.columns } # Assumption: it was 20 in 2002 and it is 22 in 2022
retirem_age = { y: 58 + 7*(y-first_year)/(last_year-first_year) for y in dfp.columns } # Assumption: it was 58 in 2002 and it is 65 in 2022

# ratio of people in working age over retired age
wr_df = pd.DataFrame(index=dfp.columns)
wr_df["working-age / retirement-age ratio"] = np.nan
for y in wr_df.index:
    wa = working_age[y]
    ra = retirem_age[y]
    wr_df.loc[y, "working-age / retirement-age ratio"] = dfp.loc[wa:ra,y].sum() / dfp.loc[ra:,y].sum()
    
display(wr_df)
print("Mean working-age/retirement-age ratio:", wr_df["working-age / retirement-age ratio"].mean().round(2))

,working-age / retirement-age ratio
2002,2.021864
2003,2.048581
2004,2.052414
2005,2.049009
2006,2.165623
2007,2.113820
2008,2.081219
2009,2.207381
2010,2.163356
2011,2.129355


Mean working-age/retirement-age ratio: 2.21


Note that I'm not consideting unemployment, assuming it is constant, and what matters is the ratio between working and retired people not if working people are actually employed or not.

I will now compute the projected retirement age that it is necessary to maintain an average working/retirement ratio of 2.2

In [7]:
wr_ratio = 2.2

# Same data plotted in two variants

fig = go.Figure()
for scenario_label, dfpproj in dfp_projs.items():
    raproj_df = pd.DataFrame(index=range(2002,2100+1))
    raproj_df["Sustainable retirement age"] = np.nan
    for y in raproj_df.index:
        wa = 22 # Assumption: people will start working at 22 on average
        for ra in range(50, 100): # I'm testing different retirement ages (ra) untill the ratio is 2.2 is reached
            if dfpproj.loc[wa:ra,y].sum() / dfpproj.loc[ra:,y].sum() > wr_ratio:
                raproj_df.loc[y, "Sustainable retirement age"] = ra
                break
        
    raproj_df["Born year retiring"] = raproj_df.index - raproj_df["Sustainable retirement age"]
    raproj_df = (
        raproj_df
        .reset_index()
        .rename(columns={"index": "Year of Retirement"})
        .astype(int)
    )
    fig.add_trace(
        go.Scatter(
        x=raproj_df["Born year retiring"],
        y=raproj_df["Sustainable retirement age"],
        mode='lines',
        text=[f'With scenario `{scenario_label}` <br> someone born in {raproj_df.at[i, "Born year retiring"]} will retire in {raproj_df.at[i, "Year of Retirement"]} at {raproj_df.at[i, "Sustainable retirement age"]}yo'
              for i in raproj_df.index],
        hoverinfo='text',
        name=scenario_label
    ))   
fig.update_layout(
    xaxis_title="Year of Birth",
    yaxis_title="Retirement Age",
    legend_title="Scenarios",
    showlegend=True,
    margin=dict(l=0, r=15, t=20, b=0),
    width=780,
    height=380,
)
print("Sustainable retirement age to maintain the same ratio of working/retired people of 2.2")
fig.show()

Sustainable retirement age to maintain the same ratio of working/retired people of 2.2


In [8]:
#save figure as html
fig.write_html("../../demographic_model_page/4_sustainable_retirement_age.html")

## Conclusions
- I made some assumptions on the age at which people starts to work and retired in the past
- I computed a ratio between working-age and retired-age people of 2.0 - 2.4 in the historical data, and picked an average value of 2.2 as the "referance" one to keep constant in the future
- Based on my projection of Italian future population, being born in 1990, I expect to retire at 68-74 year old (2058-2064) depending on the scenario (9 scenarios from Notebook#3)

## Follow-up
- Consider unemployment: this is a self-balancing secondary effect as less woking people might compensate lower unemployment too
- Run a sensitivity test on the "reference" ratio of working-age/retirement-age people as it is a very handwavy estimate
- Get more data on effective historical start age and retirement age
- Run a sensitivity test on the age at which people will start to work
- Compute the life expectancy after retirement: is it fair w.r.t. those who retired in the past years?